In [ ]:
#load in the data
#split data into training and validation data
#use training data to run different evaluation
#evaluate performance on validation data
#different classifiers could be: knearestneighbors (with different neighbors, decisiontreeclasifier)
#analyse results by comparing classifiers on different metrics, inspecting images that are classifies incorrectly etc
#select best classifier (by which rules should we select)
#create python script that takes an image, measures the features, classifies the image
#and gives its probability of being a melanoma (range from 0 to 1)

## Features

* Perimeter
* Area
* Compactness
* Assymetry
* !color

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [6]:
import assymetry.ipynb

ModuleNotFoundError: No module named 'assymetry'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=932aa97b-f6d2-4889-8f04-d6166675cc2f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>